In [17]:
from datetime import datetime, date, time, timedelta
from itertools import islice
from geopy.distance import great_circle
from operator import itemgetter
import matplotlib.pylab as plt
from pandas import DataFrame
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy
import json
import math


DELTA_GNSS = 0.05 #equivalent to about 5-6 km
DELTA_GNSS_INIT = 1 #equivalent to about 5-6 km
DELTA_POS = 0.0001 #equivalent to about 100 m

#six decimal are enough for a decimeter precision
#0.1 = 11.1 km
#0.01 = 1.11 km
#...
#0.000001 = 0.111 m
uav_latitude = 45.0059289
uav_longitude = 10.461025
uav_coo = (uav_latitude, uav_longitude)

gw_latitude = 45.004854
gw_longitude = 10.467949
gw_coo = (uav_latitude, uav_longitude)

#support variables
bts_latitude = 0
bts_longitude = 0

#Import UAV Log network Ping data from JSON file
data = []
with open('LORAWAN_TTGO.json') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except:
            print('Skip line')

#Convert JSON data from Python dict to DF
UAV_LOG = pd.json_normalize(data)

#Drop unusfull data
#UAV_LOG = UAV_LOG.drop(columns=['uav_param.status','uav_param.mode','interface','gps_param.groundspeed','gps_param.velocity.vx','uav_param.attitude.pitch','uav_param.attitude.yaw','uav_param.attitude.roll','gps_param.velocity.vy','gps_param.velocity.vz','gps_param.hdop','gps_param.vdop','gps_param.fix_type'])

#Remove duplicate entries
UAV_LOG = UAV_LOG.drop_duplicates('timestamp')
UAV_LOG = UAV_LOG.apply(pd.to_numeric, errors='ignore')

#Remove false flights
#UAV_bench = UAV_LOG['uav_param.battery']>=10.0
#UAV_LOG = UAV_LOG[UAV_bench]
UAV_LOG.iloc[78]
#Remove no GPS fix data
#GPS_Null = UAV_LOG['gps_param.satellites']>=6
#UAV_LOG = UAV_LOG[GPS_Null]

#Sort by time for time clustering
#UAV_LOG = UAV_LOG.sort_values(by='timestamp')

timestamp                        2022-07-19T07:30:53.640813139Z
gateways      [{'gateway_ids': {'gateway_id': 'mtik-riosalic...
rssi                                                        -72
snr                                                        14.0
latitude                                              45.006703
longitude                                             10.457515
altitude                                                     67
hdop                                                        0.8
satellites                                                    9
Name: 78, dtype: object

In [19]:
UAV_LOG
uav_coo = []

for log in range(len(UAV_LOG)):
    uav_coo.append((UAV_LOG.iloc[log]['timestamp'], UAV_LOG.iloc[log]['latitude'], UAV_LOG.iloc[log]['longitude'], UAV_LOG.iloc[log]['altitude'], UAV_LOG.iloc[log]['rssi'], UAV_LOG.iloc[log]['snr']))
    #print(BTS.iloc[0])
    #print(DF.iloc[log])
uav_coo

[('2022-07-19T07:03:47.036096458Z',
  45.00482529430539,
  10.467922119374407,
  16,
  -31,
  13.0),
 ('2022-07-19T07:04:07.118812212Z',
  45.00482529430539,
  10.467922119374407,
  16,
  -32,
  13.8),
 ('2022-07-19T07:04:27.211992887Z',
  45.00482529430539,
  10.467922119374407,
  16,
  -31,
  11.0),
 ('2022-07-19T07:04:47.318032429Z',
  45.00482529430539,
  10.467922119374407,
  15,
  -32,
  13.2),
 ('2022-07-19T07:05:07.434762820Z',
  45.004836023142104,
  10.467836288680815,
  14,
  -37,
  13.5),
 ('2022-07-19T07:05:27.478161442Z',
  45.00486820965219,
  10.467900661701009,
  13,
  -43,
  10.5),
 ('2022-07-19T07:05:47.522657975Z',
  45.0048789384889,
  10.467900661701009,
  12,
  -41,
  13.5),
 ('2022-07-19T07:06:07.615299108Z',
  45.0048789384889,
  10.467900661701009,
  12,
  -42,
  13.2),
 ('2022-07-19T07:06:27.698910728Z',
  45.0048789384889,
  10.467900661701009,
  12,
  -41,
  11.2),
 ('2022-07-19T07:06:47.757990592Z',
  45.0048789384889,
  10.467900661701009,
  12,
  -41,
  